# Модуль таблицы

In [28]:
import re

class Table:
    def __init__(self, data=None, set_column_types_auto = False):
        self.data = data or {}
        self.column_types = {}

        if set_column_types_auto:
          self.set_column_types_auto()

    def get_column_types(self):
        if self.column_types == {}:
          self.set_column_types_auto()
        return self.column_types

    def set_column_types_auto(self):
        def transform_type_name(type_name):
          result = re.search(r"<class '(.*?)'>", type_name)
          # Проверка, был ли найден результат
          if result:
              return result.group(1)
          return "str"

        self.column_types = {col: transform_type_name(str(type(self.data[col][0]))) if len(self.data[col]) > 0 else str for col in self.data.keys()}

    def get_rows_by_number(self, start, stop=None, copy_table=False):
        stop = stop if stop is not None else start + 1

        rows = {col: values[start:stop] for col, values in self.data.items()}
        return Table(rows) if copy_table else rows

    def get_rows_by_index(self, *values, copy_table=False):
        indices = [i for i, val in enumerate(self.data[next(iter(self.data))]) if val in values]
        rows = {col: [values[i] for i in indices] for col, values in self.data.items()}
        return Table(rows) if copy_table else rows

    def set_column_types(self, types_dict, by_number=True):
      for key, value in types_dict.items():
          col = list(self.data.keys())[key] if by_number and isinstance(key, int) else key
          if col in self.data:
              self.column_types[col] = value
              self.data[col] = [self._convert_type(data_value, value) for data_value in self.data[col]]
          else:
              raise KeyError(f"Колонка '{col}' не найдена в таблице.")

    def get_values(self, column=0):
        col = list(self.data.keys())[column] if isinstance(column, int) else column
        return [self._convert_type(value, self.column_types[col]) for value in self.data[col]]

    def get_value(self, column=0):
        values = self.get_values(column)
        return values[0]

    def set_values(self, values, column=0):
        col = list(self.data.keys())[column] if isinstance(column, int) else column
        self.data[col] = [self._convert_type(value, self.column_types[col]) for value in values]

    def set_value(self, value, column=0):
        col = list(self.data.keys())[column] if type(column) == int else column
        self.data[col] = [self._convert_type(value, self.column_types[col])]

    def print_table(self):
        print("\t".join(self.data.keys()))
        for row in zip(*self.data.values()):
            print("\t".join(map(str, row)))

    def _convert_type(self, value, type_str):
        converters = {
        int: int,
        float: float,
        bool: lambda x: str(x).lower() in ("true", "1"),
        str: str,
        "int": int,
        "float": float,
        "bool": lambda x: str(x).lower() in ("true", "1"),
        "str": str,
        }
        return converters[type_str](value)

# Модули работы с файлами

In [29]:
import csv
import pickle
import os

In [30]:
class CSVHandler:
    @staticmethod
    def load_table(*file_paths, set_column_types_auto = False):
        if not file_paths:
            raise ValueError("At least one file path must be provided.")

        all_columns = None
        all_data = []

        for file_path in file_paths:
            if not os.path.exists(file_path):
                raise FileNotFoundError(f"File not found: {file_path}")

            with open(file_path, 'r', newline='', encoding='utf-8') as file:
                reader = csv.reader(file)
                columns = next(reader, [])
                if not columns:
                    raise ValueError(f"File {file_path} contains no data or column headers.")

                # Проверяем соответствие столбцов с предыдущими файлами
                if all_columns is None:
                    all_columns = columns
                elif columns != all_columns:
                    raise ValueError(f"Column structure mismatch between files: {file_path}")

                data = list(reader)
                all_data.extend(data)

        table = {column: [] for column in all_columns}
        for row in all_data:
            for column, value in zip(all_columns, row):
                table[column].append(value)

        table_obj = Table(table)
        if set_column_types_auto:
          table_obj.set_column_types_auto()

        return table_obj

    @staticmethod
    def save_table(table, file_path="table.csv"):
        if not isinstance(table, Table):
            raise ValueError("Input must be an instance of Table.")

        with open(file_path, 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(table.data.keys())
            writer.writerows(zip(*table.data.values()))

In [31]:
class PickleHandler:
    @staticmethod
    def load_table(*file_paths, set_column_types_auto = False):
        if not file_paths:
            raise ValueError("At least one file path must be provided.")

        all_data = None

        for file_path in file_paths:
            if not os.path.exists(file_path):
                raise FileNotFoundError(f"File not found: {file_path}")

            with open(file_path, 'rb') as file:
                data = pickle.load(file)

            # Проверяем, что структура данных одинакова
            if not isinstance(data, dict) or not all(isinstance(v, list) for v in data.values()):
                raise ValueError(f"Invalid file format in file: {file_path}")

            if all_data is None:
                all_data = data
            elif all(data.keys() == all_data.keys() for data in [data, all_data]):
                for key, value in data.items():
                    all_data[key].extend(value)
            else:
                raise ValueError(f"Column structure mismatch between files: {file_path}")

        table_obj = Table(all_data)
        if set_column_types_auto:
          table_obj.set_column_types_auto()
        return table_obj

    @staticmethod
    def save_table(table, file_path="table.pkl"):
        if not isinstance(table, Table):
            raise TypeError("Input must be an instance of Table.")

        with open(file_path, 'wb') as file:
            pickle.dump(table.data, file)

In [32]:
class TextHandler:
    @staticmethod
    def save_table(table, file_path="table.txt"):
        if not isinstance(table, Table):
            raise TypeError("Input must be an instance of Table.")

        with open(file_path, 'w', encoding='utf-8') as file:
            file.write("\t".join(table.data.keys()) + "\n")
            for row in zip(*table.data.values()):
                file.write("\t".join(map(str, row)) + "\n")

# Тестирование модулей управления файлами

In [33]:
from copy import deepcopy

def test_csv_handler():
    print("Тестирование CSVHandler...")

    # Исходные данные для теста
    data1 = Table({
        "Name": ["Alice", "Bob", "Charlie"],
        "Age": [25, 30, 35],
        "City": ["New York", "Los Angeles", "Chicago"]
    })

    data2 = Table({
        "Name": ["Marina", "James", "Kattie"],
        "Age": [25, 30, 35],
        "City": ["Chicago", "Washington DC", "California"]
    })

    # Пути к файлам
    csv_file_path_1 = "table_test_1.csv"
    csv_file_path_2 = "table_test_2.csv"

    # Сохранение таблицы в два CSV файла
    CSVHandler.save_table(data1, csv_file_path_1)
    CSVHandler.save_table(data2, csv_file_path_2)

    # Загрузка таблицы из нескольких CSV файлов
    loaded_data = CSVHandler.load_table(csv_file_path_1, csv_file_path_2, set_column_types_auto=True)
    print(f"Типы столбцов {loaded_data.column_types}")

    # Приведение типов для корректного сравнения
    loaded_data.set_column_types({"Age": int})
    print("Загруженные данные из CSV:")
    loaded_data.print_table()

    merged_data = {}
    for column in data1.data:
        merged_data[column] = data1.data[column] + data2.data[column]
    # Проверка корректности данных
    assert deepcopy(loaded_data.data) == deepcopy(merged_data), "Ошибка: загруженные данные не совпадают с исходными!"
    print("Тестирование CSVHandler прошло успешно!")

# Тестирование для PickleHandler с несколькими файлами
def test_pickle_handler():
    print("Тестирование PickleHandler...")

    # Исходные данные для теста
    data1 = Table({
        "Name": ["Alice", "Bob", "Charlie"],
        "Age": [25, 30, 35],
        "City": ["New York", "Los Angeles", "Chicago"]
    })

    data2 = Table({
        "Name": ["Marina", "James", "Kattie"],
        "Age": [25, 30, 35],
        "City": ["Chicago", "Washington DC", "California"]
    })

    # Пути к файлам
    pickle_file_path_1 = "table_test_1.pkl"
    pickle_file_path_2 = "table_test_2.pkl"

    # Сохранение таблицы в два Pickle файла
    PickleHandler.save_table(data1, pickle_file_path_1)
    PickleHandler.save_table(data2, pickle_file_path_2)

    # Загрузка таблицы из нескольких Pickle файлов
    loaded_data = PickleHandler.load_table(pickle_file_path_1, pickle_file_path_2, set_column_types_auto=True)
    print(f"Типы столбцов {loaded_data.column_types}")

    # Приведение типов для корректного сравнения
    loaded_data.set_column_types({"Age": int})
    print("Загруженные данные из Pickle:")
    loaded_data.print_table()

    merged_data = {}
    for column in data1.data:
        merged_data[column] = data1.data[column] + data2.data[column]
    # Проверка корректности данных
    assert deepcopy(loaded_data.data) == deepcopy(merged_data), "Ошибка: загруженные данные не совпадают с исходными!"
    print("Тестирование PickleHandler прошло успешно!")

# Тестирование для TextHandler
def test_text_handler():
    print("Тестирование TextHandler...")
    data = Table({
        "Name": ["Alice", "Bob", "Charlie"],
        "Age": [25, 30, 35],
        "City": ["New York", "Los Angeles", "Chicago"]
    })

    # Путь к файлу
    text_file_path = "table_test.txt"

    # Сохранение таблицы в текстовый файл
    TextHandler.save_table(data, text_file_path)

    print("Тестирование TextHandler прошло успешно!")


In [34]:
test_csv_handler()
test_pickle_handler()
test_text_handler()

Тестирование CSVHandler...
Типы столбцов {'Name': 'str', 'Age': 'str', 'City': 'str'}
Загруженные данные из CSV:
Name	Age	City
Alice	25	New York
Bob	30	Los Angeles
Charlie	35	Chicago
Marina	25	Chicago
James	30	Washington DC
Kattie	35	California
Тестирование CSVHandler прошло успешно!
Тестирование PickleHandler...
Типы столбцов {'Name': 'str', 'Age': 'int', 'City': 'str'}
Загруженные данные из Pickle:
Name	Age	City
Alice	25	New York
Bob	30	Los Angeles
Charlie	35	Chicago
Marina	25	Chicago
James	30	Washington DC
Kattie	35	California
Тестирование PickleHandler прошло успешно!
Тестирование TextHandler...
Тестирование TextHandler прошло успешно!


# Тестирование модулей управления таблицами

In [35]:
# Тестирование функций класса Table
def test_table_operations():
    print("Тестирование операций класса Table...")

    data = Table({
        "Name": ["Alice", "Bob", "Charlie"],
        "Age": [25, 30, 35],
        "City": ["New York", "Los Angeles", "Chicago"]
    }, True)

    print(data.column_types)
     # get_rows_by_number
    print("\nТест get_rows_by_number (copy_table=True):")
    row = data.get_rows_by_number(1, 3, copy_table=True)
    row.set_column_types_auto()
    row.print_table()

    # get_rows_by_index
    print("\nТест get_rows_by_index (copy_table=True):")
    indexed_rows = data.get_rows_by_index("Bob", "Charlie", copy_table=True)
    indexed_rows.print_table()

    # get_rows_by_number
    print("\nТест get_rows_by_number (copy_table=True):")
    row = data.get_rows_by_number(1, copy_table=True)
    row.set_column_types_auto()
    row.print_table()

    print("\nТест get_rows_by_number (copy_table=False):")
    row_view = data.get_rows_by_number(1, copy_table=False)
    print(row_view)

    # get_rows_by_index
    print("\nТест get_rows_by_index (copy_table=True):")
    indexed_rows = data.get_rows_by_index("Bob", copy_table=True)
    indexed_rows.print_table()

    print("\nТест get_rows_by_index (copy_table=False):")
    indexed_rows_view = data.get_rows_by_index("Bob", copy_table=False)
    print(indexed_rows_view)

    # column_types
    print("\nТест column_types:")
    print("Типы колонок:", data.column_types)

    # set_column_types
    print("\nТест set_column_types:")
    data.set_column_types({"Age": "str", "Name" : "str"})
    print("Типы колонок после изменения:", data.column_types)

    data.print_table()
    # get_values
    print("\nТест get_values:")
    values = data.get_values("City")
    print("Значения из колонки 'City':", values)

    # get_value
    print("\nТест get_value:")
    single_value = row.get_value("City")
    print("Значение из колонки 'City' для одной строки:", single_value)

    # set_values
    print("\nТест set_values:")
    data.set_values(["Boston", "Houston", "Phoenix"], "City")
    print("Обновленная таблица:")
    data.print_table()

    # set_value
    print("\nТест set_value:")
    row.set_value("San Francisco", "City")
    print("Обновленная строка:")
    row.print_table()

    print("\nТестирование операций класса Table прошло успешно!")


In [36]:
test_table_operations()

Тестирование операций класса Table...
{'Name': 'str', 'Age': 'int', 'City': 'str'}

Тест get_rows_by_number (copy_table=True):
Name	Age	City
Bob	30	Los Angeles
Charlie	35	Chicago

Тест get_rows_by_index (copy_table=True):
Name	Age	City
Bob	30	Los Angeles
Charlie	35	Chicago

Тест get_rows_by_number (copy_table=True):
Name	Age	City
Bob	30	Los Angeles

Тест get_rows_by_number (copy_table=False):
{'Name': ['Bob'], 'Age': [30], 'City': ['Los Angeles']}

Тест get_rows_by_index (copy_table=True):
Name	Age	City
Bob	30	Los Angeles

Тест get_rows_by_index (copy_table=False):
{'Name': ['Bob'], 'Age': [30], 'City': ['Los Angeles']}

Тест column_types:
Типы колонок: {'Name': 'str', 'Age': 'int', 'City': 'str'}

Тест set_column_types:
Типы колонок после изменения: {'Name': 'str', 'Age': 'str', 'City': 'str'}
Name	Age	City
Alice	25	New York
Bob	30	Los Angeles
Charlie	35	Chicago

Тест get_values:
Значения из колонки 'City': ['New York', 'Los Angeles', 'Chicago']

Тест get_value:
Значение из колонки 'C